In [1]:
# # see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# %load_ext autoreload
# %autoreload 2
# %matplotlib inline
# import pdb

In [2]:
import os
import time
import os.path as op
import argparse
from data.data_pipe import get_val_pair
from torchvision import transforms as trans
from tqdm import tqdm_notebook as tqdm
import torch
import numpy as np
from IPython.display import display

from Learner import face_learner
from config import get_config
from utils import hflip_batch, cosineDim1
from data.datasets import (IJBCroppedFacesDataset, IJBCVerificationPathDataset,
                           ARVerificationAllPathDataset, IJBAVerificationDataset)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
def process_batch_original(batch, tta=False):
    if tta:
        fliped = hflip_batch(batch)
        feat_orig = learner.model.get_original_feature(batch.to(conf.device))
        feat_flip = learner.model.get_original_feature(fliped.to(conf.device))
        feat = (feat_orig + feat_flip) / 2
    else:
        feat = learner.model.get_original_feature(batch.to(conf.device))
    return feat


def process_batch_xcos(batch, tta=False):
    if tta:
        fliped = hflip_batch(batch)
        flattened_feature1, feat_map1 = learner.model(batch.to(conf.device))
        flattened_feature2, feat_map2 = learner.model(fliped.to(conf.device))
        feat_map = (feat_map1 + feat_map2) / 2
        flattened_feature = (flattened_feature1 + flattened_feature2) / 2
    else:
        flattened_feature, feat_map = learner.model(batch.to(conf.device))
    return flattened_feature, feat_map

## Init Learner and conf

In [4]:
conf = get_config(training=False)
conf.batch_size=20 # Why bs_size can only be the number that divide 6000 well?
learner = face_learner(conf, inference=True)

{'detachAttentionGradient': True, 'fixed_str': 'ir_se50.pth', 'pretrainedMdl': 'ir_se50.pth', 'data_path': PosixPath('data'), 'work_path': PosixPath('work_space'), 'model_path': PosixPath('work_space/models'), 'log_path': PosixPath('work_space/log'), 'save_path': PosixPath('work_space/save'), 'exp_title': 'xCos', 'exp_comment': 'expMS1M_detached', 'input_size': [112, 112], 'embedding_size': 1568, 'use_mobilfacenet': False, 'modelType': 'ArcFace', 'net_depth': 50, 'drop_ratio': 0.6, 'net_mode': 'ir_se', 'device': device(type='cuda', index=0), 'test_transform': Compose(
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
), 'data_mode': 'emore', 'vgg_folder': PosixPath('data/faces_vgg_112x112'), 'ms1m_folder': PosixPath('data/faces_ms1m_112x112'), 'emore_folder': PosixPath('data/faces_emore'), 'batch_size': 20, 'USE_SOFTMAX': True, 'SOFTMAX_T': 1, 'facebank_path': PosixPath('data/facebank'), 'threshold': 1.5, 'threshold_xCos': 0.2338, 'face_limit': 10, 'min_face_size'

# Process IJB-B data and save them

In [6]:
loader = torch.utils.data.DataLoader(
    IJBCroppedFacesDataset('data/IJB_release/IJBB'),
    batch_size=1
)

## Original model

In [7]:
learner.load_state(conf, 'ir_se50.pth', model_only=True, from_save_folder=True, strict=False, model_atten=False)
learner.model.eval()
learner.model.returnGrid = True  # Remember to reset this before return!
learner.model_attention.eval()

dst_dir = 'work_space/IJB_features/IJBB_ir_se50'
os.makedirs(op.join(dst_dir), exist_ok=True)
os.makedirs(op.join(dst_dir, 'loose_crop'), exist_ok=True)
with torch.no_grad():
    for i, batch in enumerate(loader):
        tensor = batch['tensor']
        src_path = batch['path']
        if i % 200 == 0:
            print(f'Processing {src_path[0]}')
        target_path = op.join(dst_dir, *src_path[0].split('/')[-2:]) + '.npy'
        if op.exists(target_path):
            if i % 200 == 0:
                print(f"Skipping {target_path} because it exists.")
            continue
        feat = process_batch_original(tensor, tta=True)
        np.save(target_path, feat.cpu().numpy())

Processing data/IJB_release/IJBB/loose_crop/1.jpg
Processing data/IJB_release/IJBB/loose_crop/201.jpg
Processing data/IJB_release/IJBB/loose_crop/401.jpg
Processing data/IJB_release/IJBB/loose_crop/601.jpg
Processing data/IJB_release/IJBB/loose_crop/801.jpg
Processing data/IJB_release/IJBB/loose_crop/1001.jpg
Processing data/IJB_release/IJBB/loose_crop/1201.jpg
Processing data/IJB_release/IJBB/loose_crop/1401.jpg
Processing data/IJB_release/IJBB/loose_crop/1601.jpg
Processing data/IJB_release/IJBB/loose_crop/1801.jpg
Processing data/IJB_release/IJBB/loose_crop/2001.jpg
Processing data/IJB_release/IJBB/loose_crop/2201.jpg
Processing data/IJB_release/IJBB/loose_crop/2401.jpg
Processing data/IJB_release/IJBB/loose_crop/2601.jpg
Processing data/IJB_release/IJBB/loose_crop/2801.jpg
Processing data/IJB_release/IJBB/loose_crop/3001.jpg
Processing data/IJB_release/IJBB/loose_crop/3201.jpg
Processing data/IJB_release/IJBB/loose_crop/3401.jpg
Processing data/IJB_release/IJBB/loose_crop/3601.jpg


Processing data/IJB_release/IJBB/loose_crop/30601.jpg
Processing data/IJB_release/IJBB/loose_crop/30801.jpg
Processing data/IJB_release/IJBB/loose_crop/31001.jpg
Processing data/IJB_release/IJBB/loose_crop/31201.jpg
Processing data/IJB_release/IJBB/loose_crop/31401.jpg
Processing data/IJB_release/IJBB/loose_crop/31601.jpg
Processing data/IJB_release/IJBB/loose_crop/31801.jpg
Processing data/IJB_release/IJBB/loose_crop/32001.jpg
Processing data/IJB_release/IJBB/loose_crop/32201.jpg
Processing data/IJB_release/IJBB/loose_crop/32401.jpg
Processing data/IJB_release/IJBB/loose_crop/32601.jpg
Processing data/IJB_release/IJBB/loose_crop/32801.jpg
Processing data/IJB_release/IJBB/loose_crop/33001.jpg
Processing data/IJB_release/IJBB/loose_crop/33201.jpg
Processing data/IJB_release/IJBB/loose_crop/33401.jpg
Processing data/IJB_release/IJBB/loose_crop/33601.jpg
Processing data/IJB_release/IJBB/loose_crop/33801.jpg
Processing data/IJB_release/IJBB/loose_crop/34001.jpg
Processing data/IJB_release/

Processing data/IJB_release/IJBB/loose_crop/61001.jpg
Processing data/IJB_release/IJBB/loose_crop/61201.jpg
Processing data/IJB_release/IJBB/loose_crop/61401.jpg
Processing data/IJB_release/IJBB/loose_crop/61601.jpg
Processing data/IJB_release/IJBB/loose_crop/61801.jpg
Processing data/IJB_release/IJBB/loose_crop/62001.jpg
Processing data/IJB_release/IJBB/loose_crop/62201.jpg
Processing data/IJB_release/IJBB/loose_crop/62401.jpg
Processing data/IJB_release/IJBB/loose_crop/62601.jpg
Processing data/IJB_release/IJBB/loose_crop/62801.jpg
Processing data/IJB_release/IJBB/loose_crop/63001.jpg
Processing data/IJB_release/IJBB/loose_crop/63201.jpg
Processing data/IJB_release/IJBB/loose_crop/63401.jpg
Processing data/IJB_release/IJBB/loose_crop/63601.jpg
Processing data/IJB_release/IJBB/loose_crop/63801.jpg
Processing data/IJB_release/IJBB/loose_crop/64001.jpg
Processing data/IJB_release/IJBB/loose_crop/64201.jpg
Processing data/IJB_release/IJBB/loose_crop/64401.jpg
Processing data/IJB_release/

Processing data/IJB_release/IJBB/loose_crop/91401.jpg
Processing data/IJB_release/IJBB/loose_crop/91601.jpg
Processing data/IJB_release/IJBB/loose_crop/91801.jpg
Processing data/IJB_release/IJBB/loose_crop/92001.jpg
Processing data/IJB_release/IJBB/loose_crop/92201.jpg
Processing data/IJB_release/IJBB/loose_crop/92401.jpg
Processing data/IJB_release/IJBB/loose_crop/92601.jpg
Processing data/IJB_release/IJBB/loose_crop/92801.jpg
Processing data/IJB_release/IJBB/loose_crop/93001.jpg
Processing data/IJB_release/IJBB/loose_crop/93201.jpg
Processing data/IJB_release/IJBB/loose_crop/93401.jpg
Processing data/IJB_release/IJBB/loose_crop/93601.jpg
Processing data/IJB_release/IJBB/loose_crop/93801.jpg
Processing data/IJB_release/IJBB/loose_crop/94001.jpg
Processing data/IJB_release/IJBB/loose_crop/94201.jpg
Processing data/IJB_release/IJBB/loose_crop/94401.jpg
Processing data/IJB_release/IJBB/loose_crop/94601.jpg
Processing data/IJB_release/IJBB/loose_crop/94801.jpg
Processing data/IJB_release/

Processing data/IJB_release/IJBB/loose_crop/121401.jpg
Processing data/IJB_release/IJBB/loose_crop/121601.jpg
Processing data/IJB_release/IJBB/loose_crop/121801.jpg
Processing data/IJB_release/IJBB/loose_crop/122001.jpg
Processing data/IJB_release/IJBB/loose_crop/122201.jpg
Processing data/IJB_release/IJBB/loose_crop/122401.jpg
Processing data/IJB_release/IJBB/loose_crop/122601.jpg
Processing data/IJB_release/IJBB/loose_crop/122801.jpg
Processing data/IJB_release/IJBB/loose_crop/123001.jpg
Processing data/IJB_release/IJBB/loose_crop/123201.jpg
Processing data/IJB_release/IJBB/loose_crop/123401.jpg
Processing data/IJB_release/IJBB/loose_crop/123601.jpg
Processing data/IJB_release/IJBB/loose_crop/123801.jpg
Processing data/IJB_release/IJBB/loose_crop/124001.jpg
Processing data/IJB_release/IJBB/loose_crop/124201.jpg
Processing data/IJB_release/IJBB/loose_crop/124401.jpg
Processing data/IJB_release/IJBB/loose_crop/124601.jpg
Processing data/IJB_release/IJBB/loose_crop/124801.jpg
Processing

Processing data/IJB_release/IJBB/loose_crop/151201.jpg
Processing data/IJB_release/IJBB/loose_crop/151401.jpg
Processing data/IJB_release/IJBB/loose_crop/151601.jpg
Processing data/IJB_release/IJBB/loose_crop/151801.jpg
Processing data/IJB_release/IJBB/loose_crop/152001.jpg
Processing data/IJB_release/IJBB/loose_crop/152201.jpg
Processing data/IJB_release/IJBB/loose_crop/152401.jpg
Processing data/IJB_release/IJBB/loose_crop/152601.jpg
Processing data/IJB_release/IJBB/loose_crop/152801.jpg
Processing data/IJB_release/IJBB/loose_crop/153001.jpg
Processing data/IJB_release/IJBB/loose_crop/153201.jpg
Processing data/IJB_release/IJBB/loose_crop/153401.jpg
Processing data/IJB_release/IJBB/loose_crop/153601.jpg
Processing data/IJB_release/IJBB/loose_crop/153801.jpg
Processing data/IJB_release/IJBB/loose_crop/154001.jpg
Processing data/IJB_release/IJBB/loose_crop/154201.jpg
Processing data/IJB_release/IJBB/loose_crop/154401.jpg
Processing data/IJB_release/IJBB/loose_crop/154601.jpg
Processing

Processing data/IJB_release/IJBB/loose_crop/181001.jpg
Processing data/IJB_release/IJBB/loose_crop/181201.jpg
Processing data/IJB_release/IJBB/loose_crop/181401.jpg
Processing data/IJB_release/IJBB/loose_crop/181601.jpg
Processing data/IJB_release/IJBB/loose_crop/181801.jpg
Processing data/IJB_release/IJBB/loose_crop/182001.jpg
Processing data/IJB_release/IJBB/loose_crop/182201.jpg
Processing data/IJB_release/IJBB/loose_crop/182401.jpg
Processing data/IJB_release/IJBB/loose_crop/182601.jpg
Processing data/IJB_release/IJBB/loose_crop/182801.jpg
Processing data/IJB_release/IJBB/loose_crop/183001.jpg
Processing data/IJB_release/IJBB/loose_crop/183201.jpg
Processing data/IJB_release/IJBB/loose_crop/183401.jpg
Processing data/IJB_release/IJBB/loose_crop/183601.jpg
Processing data/IJB_release/IJBB/loose_crop/183801.jpg
Processing data/IJB_release/IJBB/loose_crop/184001.jpg
Processing data/IJB_release/IJBB/loose_crop/184201.jpg
Processing data/IJB_release/IJBB/loose_crop/184401.jpg
Processing

Processing data/IJB_release/IJBB/loose_crop/210801.jpg
Processing data/IJB_release/IJBB/loose_crop/211001.jpg
Processing data/IJB_release/IJBB/loose_crop/211201.jpg
Processing data/IJB_release/IJBB/loose_crop/211401.jpg
Processing data/IJB_release/IJBB/loose_crop/211601.jpg
Processing data/IJB_release/IJBB/loose_crop/211801.jpg
Processing data/IJB_release/IJBB/loose_crop/212001.jpg
Processing data/IJB_release/IJBB/loose_crop/212201.jpg
Processing data/IJB_release/IJBB/loose_crop/212401.jpg
Processing data/IJB_release/IJBB/loose_crop/212601.jpg
Processing data/IJB_release/IJBB/loose_crop/212801.jpg
Processing data/IJB_release/IJBB/loose_crop/213001.jpg
Processing data/IJB_release/IJBB/loose_crop/213201.jpg
Processing data/IJB_release/IJBB/loose_crop/213401.jpg
Processing data/IJB_release/IJBB/loose_crop/213601.jpg
Processing data/IJB_release/IJBB/loose_crop/213801.jpg
Processing data/IJB_release/IJBB/loose_crop/214001.jpg
Processing data/IJB_release/IJBB/loose_crop/214201.jpg
Processing

## Our model

In [8]:
def run_ours_IJBC(model_name):
    learner.load_state(
        conf, f'{model_name}.pth',
        model_only=True, from_save_folder=True, strict=True, model_atten=True)
    learner.model.eval()
    learner.model.returnGrid = True  # Remember to reset this before return!
    learner.model_attention.eval()

    dst_dir = f'work_space/IJB_features/IJBB_{model_name}/'
    os.makedirs(op.join(dst_dir), exist_ok=True)
    os.makedirs(op.join(dst_dir, 'loose_crop'), exist_ok=True)

    with torch.no_grad():
        for i, batch in enumerate(loader):
            tensor = batch['tensor']
            src_path = batch['path']
            target_path = op.join(dst_dir, *src_path[0].split('/')[-2:]) + '.npz'
            if i % 2000 == 0:
                print(f'Processing {src_path[0]} \nSaving to {target_path}')
            if op.exists(target_path):
                if i % 2000 == 0:
                    print(f"Skipping {target_path} because it exists.")
                continue
            flattened_feature, feat_map = process_batch_xcos(tensor, tta=True)
            np.savez(target_path, flattened_feature=flattened_feature.cpu().numpy(),
                     feat_map=feat_map.cpu().numpy())


model_names = [
#     '2019-09-01-15-30_accuracy:0.9946666666666667_step:218346_CosFace',
    '2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace',
#     '2019-08-25-14-35_accuracy:0.9931666666666666_step:218349_None',
    '2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None'
]
for model_name in model_names:
    print(f'-------- Runing on model {model_name} --------')
    run_ours_IJBC(model_name)

-------- Runing on model 2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace --------
Processing data/IJB_release/IJBB/loose_crop/1.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/1.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/2001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/2001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/4001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/4001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/6001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/6001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/8001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/8001.jpg.n

Processing data/IJB_release/IJBB/loose_crop/90001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/90001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/92001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/92001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/94001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/94001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/96001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/96001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/98001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/98001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/100001.jpg 
Saving to work_space/IJB

Processing data/IJB_release/IJBB/loose_crop/180001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/180001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/182001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/182001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/184001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/184001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/186001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/186001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/188001.jpg 
Saving to work_space/IJB_features/IJBB_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/loose_crop/188001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/190001.jpg 
Saving to work

Processing data/IJB_release/IJBB/loose_crop/42001.jpg 
Saving to work_space/IJB_features/IJBB_2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/loose_crop/42001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/44001.jpg 
Saving to work_space/IJB_features/IJBB_2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/loose_crop/44001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/46001.jpg 
Saving to work_space/IJB_features/IJBB_2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/loose_crop/46001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/48001.jpg 
Saving to work_space/IJB_features/IJBB_2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/loose_crop/48001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/50001.jpg 
Saving to work_space/IJB_features/IJBB_2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/loose_crop/50001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/52001.jpg 
Saving to work_space/IJB_features/IJBB_2

Processing data/IJB_release/IJBB/loose_crop/134001.jpg 
Saving to work_space/IJB_features/IJBB_2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/loose_crop/134001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/136001.jpg 
Saving to work_space/IJB_features/IJBB_2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/loose_crop/136001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/138001.jpg 
Saving to work_space/IJB_features/IJBB_2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/loose_crop/138001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/140001.jpg 
Saving to work_space/IJB_features/IJBB_2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/loose_crop/140001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/142001.jpg 
Saving to work_space/IJB_features/IJBB_2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/loose_crop/142001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/144001.jpg 
Saving to work_space/IJB_feat

Processing data/IJB_release/IJBB/loose_crop/224001.jpg 
Saving to work_space/IJB_features/IJBB_2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/loose_crop/224001.jpg.npz
Processing data/IJB_release/IJBB/loose_crop/226001.jpg 
Saving to work_space/IJB_features/IJBB_2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/loose_crop/226001.jpg.npz
